In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from srim import TRIM, SR, Ion, Layer, Target
from srim.output import Results, Collision
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [3]:
#retrieve data from initial recoil
initial_ion_data= []
with open('C://Users/hep/Desktop/Srim_data_(nathan)/50 keV_1 simulation4_(exyz)/COLLISON.txt', 'r') as file:
    for line in file:
          if line.strip().endswith('<== Start of New Cascade  ³'):
               data_points = line.strip().split('³')
               numpy_array = np.array(data_points)
               initial_ion_data.append(numpy_array)

In [4]:
#organize the information from each collision
remove_quotes = np.vectorize(lambda x: x.replace("'", ""))
cleaned_ion_array = remove_quotes(initial_ion_data)

In [5]:
#put initial ion energy and xyz coordinates each into their own numpy array
init_ion_E=[]
for i in range(len(cleaned_ion_array)):
    init_ion_E.append(cleaned_ion_array[i][2])

init_ion_x=[]
for i in range(len(cleaned_ion_array)):
    init_ion_x.append(cleaned_ion_array[i][3])

init_ion_y=[]
for i in range(len(cleaned_ion_array)):
    init_ion_y.append(cleaned_ion_array[i][4])

init_ion_z=[]
for i in range(len(cleaned_ion_array)):
    init_ion_z.append(cleaned_ion_array[i][5])

init_ion_E= np.array(init_ion_E,dtype=np.float64)
init_ion_x= np.array(init_ion_x,dtype=np.float64)
init_ion_y= np.array(init_ion_y,dtype=np.float64)
init_ion_z= np.array(init_ion_z,dtype=np.float64)

In [6]:
# read the data from the recoils into a pandas dataframe
widths2= [1,7,4,12,11,11,11]
df = pd.read_fwf('C://Users/hep/Desktop/Srim_data_(nathan)/50 keV_1 simulation4_(exyz)/COLLISON.txt', skiprows=28, skipfooter=1, delimeter=' ', widths= widths2, header=None, engine='python',encoding='latin-1')

In [7]:
#Consider only the rows corresponding to the data
df_filtered = df[df[0] == 'Û']

In [8]:
#check to see the info was retrieved
df_filtered.info

<bound method DataFrame.info of       0      1   2           3          4          5          6
0     Û  00001  18  20637.E-03  1803.E+01  4757.E+00  1839.E+01
6     Û  00001  18  26410.E-02  4780.E+01  7156.E+00  2373.E+01
7     Û  00002  18  12931.E-03  4593.E+01  3126.E+01  4914.E+01
8     Û  00003  18  72081.E-04  4674.E+01  2871.E+01  4845.E+01
9     Û  00004  18  14029.E-03  4249.E+01  2088.E+01  4013.E+01
...  ..    ...  ..         ...        ...        ...        ...
3985  Û  00001  18  60686.E-04  1390.E+03  1867.E+02  3807.E+02
3991  Û  00001  18  25051.E-03  1389.E+03  1868.E+02  3804.E+02
3992  Û  00002  18  53437.E-04  1388.E+03  1872.E+02  3804.E+02
3993  Û  00003  18  52131.E-04  1389.E+03  1869.E+02  3804.E+02
3999  Û  00001  18  12263.E-03  1385.E+03  1949.E+02  3761.E+02

[3500 rows x 7 columns]>

In [9]:
#group the pandas dataframe into each cascade event
recoil_count= df_filtered[1].astype(int)
reset_condition = (recoil_count <= recoil_count.shift(fill_value=0)).cumsum()
grouped = df_filtered.groupby(reset_condition)

In [10]:
#print out the groups to make sure they correspond to the cascade events
for group_id, group_data in grouped:
    print(f"Group {group_id}:")
    print(group_data)

Group 0:
   0      1   2           3          4          5          6
0  Û  00001  18  20637.E-03  1803.E+01  4757.E+00  1839.E+01
Group 1:
    0      1   2           3          4          5          6
6   Û  00001  18  26410.E-02  4780.E+01  7156.E+00  2373.E+01
7   Û  00002  18  12931.E-03  4593.E+01  3126.E+01  4914.E+01
8   Û  00003  18  72081.E-04  4674.E+01  2871.E+01  4845.E+01
9   Û  00004  18  14029.E-03  4249.E+01  2088.E+01  4013.E+01
10  Û  00005  18  43385.E-03  4313.E+01  1717.E+01  3578.E+01
11  Û  00006  18  73844.E-04  3964.E+01  2013.E+01  3790.E+01
12  Û  00007  18  22152.E-03  4083.E+01  2033.E+01  3696.E+01
13  Û  00008  18  50718.E-04  3369.E+01  1813.E+01  3374.E+01
14  Û  00009  18  93247.E-04  3721.E+01  1826.E+01  3373.E+01
15  Û  00010  18  13829.E-02  4646.E+01  1660.E+01  3329.E+01
16  Û  00011  18  13067.E-02  4597.E+01  1461.E+01  3239.E+01
17  Û  00012  18  14440.E-03  4019.E+01  3359.E+00  2280.E+01
18  Û  00013  18  58202.E-04  4414.E+01  6978.E-01  22